## Using Tools in Llama 3.1 to provide real-time information in an LLM chat

Llama 3.1 was just released this week, and supports 'tools'.  Tools are simple functions that you can write and provide to the LLM to answer particular questions.   This can be very valuable in an agentic workflow built for a highly specialized purpose, where you can use the LLM to determine the intent of a user's question in a chat, extract the relevant parameters from the chat and pipe them into the function you have written in order to get the required info.

Let's look at a simple example.  I'm going to create a workflow to find out the current temperature in a particular place of interest.

### Creating a tool to get the current temperature

In this code I am writing a simple Python function which will ping an API to get the current temperature in a place.

In [1]:
import requests

# function to get the current temperature in a place
def get_current_temperature(place: str) -> str:
  base_url = f"https://wttr.in/{place}?format=j1"
  response = requests.get(base_url)
  data = response.json()
  return f"The current temperature in {place} is {data['current_condition'][0]['temp_C']} degrees Celsius"

# test the function
get_current_temperature("London")

'The current temperature in London is 28 degrees Celsius'

That seems to have worked.  

### Adding the tool to the LLM workflow

Next I am going to use this function as a tool to feed to the LLM.  Not all LLMs support tools, but for those that do, the idea is as follows:
* The LLM should identify the intent of a user query and determine that the intent is appropriate to call the tool
* The LLM should extract or determine the right arguments/parameters to give to the tool
* The LLM should capture the response of the tool and feed it back to the user.  To do this it may have to override some default behaviors (for example many LLMs have default responses which inform users that they cannot provide certain real-time information).

To create this workflow, I am going to use an asynchronous chat client via Ollama - I'll leave the model as a parameter so we can try out a few models at the end.

In [2]:
import ollama
import asyncio

async def weather_chat(model: str, query: str):
  client = ollama.AsyncClient()
  # Initialize conversation with a query
  messages = [{'role': 'user', 'content': query}]

  # First API call: Send the query and function description to the model
  # use the tools parameter to define the tool so the LLM can understand it 
  response = await client.chat(
    model = model,
    messages = messages,
    tools = [
      {
        'type': 'function',
        'function': {
          'name': 'get_current_temperature',
          'description': 'Get the temperature in a place',
          'parameters': {
            'type': 'object',
            'properties': {
              'place': {
                'type': 'string',
                'description': 'The place for which the temperature is requested',
              }
            },
            'required': ['place'],
          },
        },
      },
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided tool and warn if it did not
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # If it did use the tool, process tool calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'get_current_temperature': get_current_temperature,
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['place'])
      # Add function response to the conversation
      # Warn model to override any default behavior
      messages.append(
        {
          'role': 'tool',
          'content': f"""
          Ignore any other information and use only this information, if relevant, to answer the original question.  {function_response}
          """,
        }
      )

  # Second API call: Get final response from the model
  # Hopefully it will provide the information obtained using the tool
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])


### Testing using various open models and queries
First, let's try this with Google's Gemma2 model and a simple question.


In [14]:
await weather_chat('gemma2:27b', 'What is the temperature like right now in Dublin?')

ResponseError: gemma2:27b does not support tools

OK, so we can see that some models don't yet support this feature.  Let's try the small version of Llama 3.1.

In [15]:
await weather_chat('llama3.1:8b', 'What is the temperature like right now in Dublin?')

I can't provide real-time information or access external data. However, I can suggest some ways for you to find out the current temperature in Dublin.

1. Check online weather websites such as AccuWeather or Weather.com.
2. Use a search engine like Google and type "current temperature in Dublin" to get the latest updates.
3. Look up the official website of the Irish Meteorological Service (Met Éireann) for the most accurate and up-to-date information.

Please note that temperatures can vary depending on the source and location within Dublin, so it's always a good idea to check multiple sources for the most accurate information.


OK so the small version of LLama 3.1 has not been able to recognise the intent of the question and use my function, and hence defaults to it's standard behavior when asked for real time information or external data.  Let's try a larger version of Llama 3.1.

In [16]:
await weather_chat('llama3.1:70b', 'What is the temperature like right now in Dublin?')

The current temperature in Dublin is 21 degrees Celsius.


Oh, that's cool!  Llama 70b is able to handle my workflow.  So what happens if we make the question a bit less direct?

In [17]:
await weather_chat('llama3.1:70b', "What is the temperature like in Ireland's capital city?")

The current temperature in Dublin, Ireland's capital city, is 21 degrees Celsius.


Very nice! Let's push things a little further.

In [18]:
await weather_chat('llama3.1:70b', "My sister says that if I went flew into the capital City of Norway today, I should dress for a very cold temperature less than 5 degrees Celsius.  Should I trust her advice?")

Based on the information provided by the weather API, the current temperature in Oslo, Norway is 21 degrees Celsius. Therefore, your sister's advice to dress for a very cold temperature less than 5 degrees Celsius is incorrect. You should not trust her advice in this case. It would be more suitable to dress for mild or warm weather instead.


In [24]:
await weather_chat('llama3.1:70b', "Compare the temperatures of these two cities right now: Dunedin, New Zealand and Reykjavik, Iceland?")

The current temperature in Dunedin, New Zealand is 5 degrees Celsius, while the current temperature in Reykjavik, Iceland is 13 degrees Celsius. Therefore, it is currently warmer in Reykjavik, Iceland than in Dunedin, New Zealand by a difference of 8 degrees Celsius.
